In [ ]:
import os
import random
import glob 
import cv2
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # CuDNN 결정론적 옵션 설정
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 원하는 시드 값 설정
seed = 42
set_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 디바이스 설정
print(device)

In [ ]:
people_list = os.listdir('./real_test')
print(len(people_list))
print(people_list[:5])

In [ ]:
left_image_paths = glob.glob(os.path.join('real_test','pair*','L.jpg'))
right_image_paths = glob.glob(os.path.join('real_test','pair*','R.jpg'))

In [ ]:
print(len(left_image_paths))
print(left_image_paths[:10])
print(len(right_image_paths))
print(right_image_paths[:10])

In [ ]:
mtcnn = MTCNN(image_size = 112, keep_all=False).to(device) # keep_all=True면 여러 얼굴 반환 

In [ ]:
from PIL import Image
from facenet_pytorch import MTCNN, extract_face
mtcnn = MTCNN(keep_all=False,device=device) # keep_all=True면 여러 얼굴 반환

detect_count = 0 
not_detect_count = 0

for image_path in tqdm(left_image_paths+right_image_paths):
    folder,pair,file_name = image_path.split('/')
    new_image_path = os.path.join('crop_real_test',pair,file_name)
    
    if not os.path.exists(os.path.join('crop_real_test',pair)):
        os.makedirs(os.path.join('crop_real_test',pair))
        
    img = Image.open(image_path)
    face, _ = mtcnn.detect(img)

    if face is not None:
        x1, y1, x2, y2 = [int(coord) for coord in face[0]]
        cropped_face = img.crop((x1-10, y1-10, x2+10, y2))
        save_path = new_image_path
        cropped_face.save(save_path)
        detect_count+=1
    else:
        h = 230
        w = 346
        x1 = w//2-h//2
        x2 = w//2+h//2
        y1 = 0
        y2 = h
        cropped_face = img.crop((x1, y1, x2, y2))
        save_path = new_image_path
        cropped_face.save(save_path)
        not_detect_count+=1
        
print(f'detect : {detect_count}, not_detect : {not_detect_count}')

In [ ]:
crop_left_image_paths = glob.glob(os.path.join('crop_real_test','pair*','L.jpg'))
crop_right_image_paths = glob.glob(os.path.join('crop_real_test','pair*','R.jpg'))

In [ ]:
image_path = './crop_real_test'

fig,axes = plt.subplots(5,2,figsize=(6,12))

for i,idx in enumerate(range(0,5)):
    left_image = os.path.join(image_path,f'pair{idx}',f'L.jpg')
    right_image = os.path.join(image_path,f'pair{idx}',f'R.jpg')
    
    left_image_bgr = cv2.imread(left_image)
    left_image_rgb = cv2.cvtColor(left_image_bgr, cv2.COLOR_BGR2RGB)

    right_image_bgr = cv2.imread(right_image)
    right_image_rgb = cv2.cvtColor(right_image_bgr, cv2.COLOR_BGR2RGB)
    
    ax = axes[i, 0]
    ax.axis('off')
    ax.set_title(f"Label: {idx}_left")
    ax.imshow(left_image_rgb)
    
    ax = axes[i, 1]
    ax.axis('off')
    ax.set_title(f"Label: {idx}_right")
    ax.imshow(right_image_rgb)

plt.show()